# Phase 1: Scikit-learn

Tên và MSSV của từng thành viên:
- Đinh Viết Lợi - 22120188.
- Nguyễn Trần Lợi - 22120190.
- Nguyễn Nhật Long - 22120194.


---

## Nắm yêu cầu của Phase 1:
Đây là bài toán phân loại nhận biết đoạn âm thanh có chứa tiếng gà tây hay không bằng cách sử dụng thư viện scikit-learn và tuân theo quy trình: Phân tích Dữ liệu Khám phá (Exploratory Data Analysis), Phát triển Mô hình (Model Development) và Đánh giá Mô hình (Model Evaluation).

## Import các thư viện cần thiết

In [45]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve,precision_recall_curve,precision_score,recall_score,f1_score
from sklearn.impute import SimpleImputer


from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek

## Đọc dữ liệu

In [46]:
path_to_data = "."

In [47]:
with open(path_to_data + "/train.json") as f:
    train_data = json.load(f)

with open(path_to_data + "/test.json") as f:
    test_data = json.load(f)

# Chuyển  dữ liệu thành DataFrame để dễ quan sát
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)


In [48]:
# Dữ liệu huấn luỵện
train_df.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [49]:
# Dữ liệu test
test_df.head()

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30


## Khám phá dữ liệu

### Các cột có trong dữ liệu

In [50]:
train_df.columns

Index(['audio_embedding', 'is_turkey', 'vid_id',
       'end_time_seconds_youtube_clip', 'start_time_seconds_youtube_clip'],
      dtype='object')

- Dữ liệu trong `train_df` chứa các cột trên. Trong đó, các cột cần thiết để thực hiện huấn luyện là `audio_embedding`, `is_turkey`.

In [51]:
test_df.columns

Index(['audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip',
       'start_time_seconds_youtube_clip'],
      dtype='object')

- `test_df` có các cột trên. Dựa vào yêu cầu, các cột sử dụng là `vid_id`, `audio_embedding`.

## Phát triển mô hình

### Xử lý dữ liệu trước khi huấn luyện

- Vì mỗi `audio_embedding` có số lượng frame khác nhau do đó để dễ dàng trong việc huấn luyện ta thực hiện lấy trung bình của mỗi cột trong các `audio_embedding`.

In [52]:
def combined_embeddings(embeddings): # Hàm dùng để tính toán trên embeddings
    X= np.array(embeddings)
    return np.mean(X, axis=0)


In [53]:
train_X = np.stack(train_df['audio_embedding'].apply(combined_embeddings)) # Lấy trung bình của mỗi cột trong các audio_embedding
train_Y = train_df['is_turkey'].values # Lấy nhãn của dữ liệu huấn luyện

valid_idx = test_df['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_df['audio_embedding'].apply(combined_embeddings)) # Lấy trung bình của mỗi cột trong các audio_embedding

- Chuẩn hóa dữ liệu để giúp cho mô hình học dữ liệu tốt hơn.

In [54]:
scaler= StandardScaler()

# Chuyển dữ liệu thành dạng chuẩn
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)

- Chia dữ liệu huấn luyện

In [55]:
train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.7,random_state=45)

### Lựa chọn mô hình với các tham số hiệu quả nhất

In [56]:
 # SVM với tham số tốt nhất từ GridSearch
model = SVC(
            probability=True,   
            random_state=42
        )

### Huấn luyện 

In [57]:
# Huấn luyện mô hình
model.fit(train_Z, train_Y)

SVC(probability=True, random_state=42)

## Đánh giá mô hình

### Dự đoán trên tập val

In [58]:
# Dự đoán trên validation
y_pred_prob = model.predict_proba(val_Z)[:, 1]
y_pred = model.predict(val_Z)

### Đánh giá

In [59]:
# In các chỉ số đánh giá
print("AUC Score   :", roc_auc_score(val_Y, y_pred_prob))
print("Accuracy    :", accuracy_score(val_Y, y_pred))
print("Precision   :", precision_score(val_Y, y_pred))
print("Recall      :", recall_score(val_Y, y_pred))
print("F1 Score    :", f1_score(val_Y, y_pred))

AUC Score   : 0.9904747703686031
Accuracy    : 0.953405017921147
Precision   : 0.9507692307692308
Recall      : 0.9307228915662651
F1 Score    : 0.9406392694063926


## Dự đoán và lưu lại kết quả trên tập Test

### Dự đoán trên tập test

In [60]:
# Dự đoán trên tập test 
test_pred_prob = model.predict_proba(test_Z)[:, 1]
test_df['is_turkey'] = -1.0
test_df.loc[valid_idx, 'is_turkey'] = test_pred_prob  

### Lưu lại kết quả

In [61]:
# Xuất file kết quả
test_df.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('svm_result.csv', index=False)